# Simulate the ground state of a Hydrogen molecule using Variational Quantum Eigensolver (VQE) on the Quantinuum emulator

![Hydrogen molecule](https://user-images.githubusercontent.com/4041805/166981145-c33b8d1a-24d1-4776-91ee-f514b0a5ab04.jpg)

In this notebook, you'll learn how to run VQE for a $H_{2}$ molecule on the Quantinuum emulator using Qiskit on an Azure Quantum backend.

VQE is a variational algorithm for quantum chemistry that uses an optimization loop to minimize a cost function. The cost function is an energy evaluation $E = <\psi|H|\psi>$ where $|\psi (\theta)>$ is a parametric trial state that estimates the ground state of the molecule. For each evaluation, we modify the trial state until the energy reaches a minimum.

![VQE diagram](https://user-images.githubusercontent.com/4041805/166981008-023aba4c-26f8-498e-93ee-a1d9a39ddbcd.png)

For more information about running VQE using Qiskit, see: [Qiskit Textbook - VQE Molecules](https://qiskit.org/textbook/ch-applications/vqe-molecules.html#implementationnoisy).

To read more about the optimization method used in this example, see [Wikipedia - SPSA](https://en.wikipedia.org/wiki/Simultaneous_perturbation_stochastic_approximation).

Before geting started, you need to import the required packages.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import IBMQ, BasicAer, Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

First, prepare the qubit operators to get the one-body and two-body integrals that encode the Hydrogen molecule and map them onto qubits using Quantum gates.

You'll use [PySCF](https://github.com/pyscf/pyscf) to generate the molecule, and [Qiskit Chemistry](https://quantum-computing.ibm.com/lab/docs/iql/chemistry) to encode it into Fermionic operators.

In [2]:
# Create a PySCF driver an generate the molecule
driver = PySCFDriver(atom='H .0 .0 -0.3625; H .0 .0 0.3625', unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
molecule = driver.run()
# Get the total number of particles in the molecule
num_particles = molecule.num_alpha + molecule.num_beta
# Convert one-body and two-body integrals into fermionic operators
qubit_op = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals).mapping(map_type='parity')
qubit_op = Z2Symmetries.two_qubit_reduction(qubit_op, num_particles)

## 1. Simulate locally

Here, you will simulate the program locally using the Aer simulator. You can create a `QuantumInstance` with a noise model using a mock device `FakeVigo` with noise characteristics.

In [3]:
from qiskit.test.mock import FakeVigo
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

backend = AerSimulator()
device_backend = FakeVigo()
device = QasmSimulator.from_backend(device_backend)

Then, run the simulation using the VQE class.

### Simulate locally with noise and error mitigation

You can read more about error mitigation in the Qiskit textbook chapter on [Measurement Error Mitigation](https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html).

In [4]:
# Create the noise characteristics and other parameters that describe the device
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates

# Create the quantum instance to conncet to the backend
quantum_instance = QuantumInstance(backend=backend, 
                                   shots=8192, 
                                   noise_model=noise_model, 
                                   coupling_map=coupling_map,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=30)

# Calculate the exact solution using numpy
exact_solution = NumPyEigensolver(qubit_op).run()
print("Exact Result:", np.real(exact_solution.eigenvalues) + molecule.nuclear_repulsion_energy)

# Create an optimizer (using SPSA)
optimizer = SPSA(maxiter=100)

# Create the variational form
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")

# Create a VQE object that runs VQE using the above created qubit operations, variational form and optimizer
vqe = VQE(qubit_op, var_form, optimizer=optimizer)

# Run the full VQE program
ret = vqe.run(quantum_instance)

# Get and print the result
vqe_result = np.real(ret['eigenvalue'] + molecule.nuclear_repulsion_energy)
print("VQE Result on noisy simulator:", vqe_result)

Exact Result: [-1.13722138]
VQE Result on noisy simulator: -1.0980120899310355


The parameters found by the optimization loop:

In [5]:
p0 = ret.optimal_point
p0

array([ 0.88146786,  2.14943478, -0.84908566,  1.72896896, -1.57937698,
        0.00403366,  2.67581075,  0.99780415, -1.25767836,  1.28513983,
        0.99229133,  1.02959351, -0.82504384, -0.44857987, -2.32099255,
        0.30197518])

The energy was evaluated a total of `ret.cost_function_evals` times until the minimum was found.

In [6]:
ret.cost_function_evals

241

### Circuit visualization

Each energy evaluation consists of two circuits. You can visualize these circuits with Qiskit using the `vqe` instance.

In [7]:
# The VQE class generates extra unused wires.
# This function is to remove idle wires to make the visualization more readable.
# See: https://quantumcomputing.stackexchange.com/questions/25672/remove-inactive-qubits-from-qiskit-circuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from collections import OrderedDict

def remove_idle_qwires(circ):
    dag = circuit_to_dag(circ)

    idle_wires = list(dag.idle_wires())
    for w in idle_wires:
        dag._remove_idle_wire(w)
        dag.qubits.remove(w)

    dag.qregs = OrderedDict()

    return dag_to_circuit(dag)

circs = vqe._circuit_sampler._transpiled_circ_cache
circs = [remove_idle_qwires(circ) for circ in circs]
circ = circs[0]

In [8]:
circ.draw()

┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐     ┌──────────┐»
   0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├──■──┤ Ry(θ[8]) ├»
      ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤┌─┴─┐├──────────┤»
   1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├┤ X ├┤ Ry(θ[9]) ├»
      └──────────┘└──────────┘└───┘└──────────┘└──────────┘└───┘└──────────┘»
c0: 2/══════════════════════════════════════════════════════════════════════»
                                                                            »
«      ┌───────────┐     ┌───────────┐┌───────────┐┌───┐┌─┐   
«   0: ┤ Rz(θ[10]) ├──■──┤ Ry(θ[12]) ├┤ Rz(θ[14]) ├┤ H ├┤M├───
«      ├───────────┤┌─┴─┐├───────────┤├───────────┤├───┤└╥┘┌─┐
«   1: ┤ Rz(θ[11]) ├┤ X ├┤ Ry(θ[13]) ├┤ Rz(θ[15]) ├┤ H ├─╫─┤M├
«      └───────────┘└───┘└───────────┘└───────────┘└───┘ ║ └╥┘
«c0: 2/══════════════════════════════════════════════════╩══╩═
«                                                        0  1

This visualization shows the parametric trial state that is prepared and evaluated as part of VQE. The parameters, $\theta[n]$, are assigned a value for each iteration.

In [9]:
circ.assign_parameters(ret.optimal_parameters).draw()

┌──────────────────────┐┌───────────────────────┐     »
   0: ─┤ Ry(0.88146786270954) ├┤ Rz(-1.25767836106238) ├──■──»
      ┌┴──────────────────────┤└┬──────────────────────┤┌─┴─┐»
   1: ┤ Ry(0.997804153760643) ├─┤ Rz(1.28513982964721) ├┤ X ├»
      └───────────────────────┘ └──────────────────────┘└───┘»
c0: 2/═══════════════════════════════════════════════════════»
                                                             »
«      ┌───────────────────────┐┌────────────────────────┐     »
«   0: ┤ Ry(0.992291331777119) ├┤ Rz(-0.825043844596012) ├──■──»
«      └┬─────────────────────┬┘├────────────────────────┤┌─┴─┐»
«   1: ─┤ Ry(1.0295935084448) ├─┤ Rz(-0.448579872331894) ├┤ X ├»
«       └─────────────────────┘ └────────────────────────┘└───┘»
«c0: 2/════════════════════════════════════════════════════════»
«                                                              »
«      ┌───────────────────────┐ ┌──────────────────────┐      »
«   0: ┤ Ry(-2.32099254965095) ├─┤ Rz(2.14943478014898) ├───■──»
«      ├───────────────────────┤┌┴──────────────────────┴┐┌─┴─┐»
«   1: ┤ Ry(0.301975180425836) ├┤ Rz(-0.849085656093776) ├┤ X ├»
«      └───────────────────────┘└────────────────────────┘└───┘»
«c0: 2/════════════════════════════════════════════════════════»
«                                                              »
«       ┌──────────────────────┐┌─────────────────────────┐┌───┐┌─┐   
«   0: ─┤ Ry(1.72896895522764) ├┤ Rz(0.00403365782424605) ├┤ H ├┤M├───
«      ┌┴──────────────────────┤└─┬─────────────────────┬─┘├───┤└╥┘┌─┐
«   1: ┤ Ry(-1.57937697516976) ├──┤ Rz(2.6758107484427) ├──┤ H ├─╫─┤M├
«      └───────────────────────┘  └─────────────────────┘  └───┘ ║ └╥┘
«c0: 2/══════════════════════════════════════════════════════════╩══╩═
«                                                                0  1

## 2. Run on Quantinuum emulator via Azure Quantum Workspace

Now, you can connect to the Azure Quantum Workspace and run VQE on the hardware backends.

In [10]:
# Connect to the Azure Quantum Workspace via a Qiskit provider
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
    resource_id = "",
    location = ""
)

In [11]:
# Create Quantinuum emulator backend
quantinuum_emulator = provider.get_backend("quantinuum.sim.h1-1e")

### Estimate cost

You can now estimate how many credits ("HQC"/"EHQC" for Quantinuum) it will cost to run VQE. For more information about pricing, see the [Azure Quantum pricing](https://docs.microsoft.com/azure/quantum/pricing) documentation page.

In [12]:
cost = [quantinuum_emulator.estimate_cost(circ.assign_parameters(ret.optimal_parameters), shots=1000) for circ in circs]

In [13]:
for _cost in cost:
    print(_cost.estimated_total, _cost.currency_code)

16.6 EHQC
16.2 EHQC


So, given approx. 300 energy evaluations, this would give a total cost

In [14]:
num_iterations = 300
energy_eval_cost = sum(_cost.estimated_total for _cost in cost)
print(f"Single iteration: {energy_eval_cost} {_cost.currency_code}, {num_iterations} iterations: {energy_eval_cost * num_iterations} {_cost.currency_code}")

Single iteration: 32.8 EHQC, 300 iterations: 9840.0 EHQC


To get a visual on the circuit width and depth, run:

In [15]:
print("width | depth")
for circuit in circs:
    circuit = circ.assign_parameters(ret.optimal_parameters)
    circuit = remove_idle_qwires(circuit)
    print(circuit.width(), circuit.depth())

width | depth
4 13
4 13


### Run one iteration on Quantinuum emulator via Azure Quantum

It can take a long time to run a full VQE program on hardware, because each iteration puts a circuit in the queue. For demonstration purposes, you can run only the last iteration using the parameters we found with the Aer simulator.

In [16]:
# This is a bug that will be addressed in this PR: https://github.com/microsoft/qdk-python/pull/301
quantinuum_emulator.configuration().max_shots = None

In [17]:
# Create Quantum Instance to connect to the backend
quantum_instance = QuantumInstance(backend=quantinuum_emulator,
                                   shots=8192)
# Unset qjob config to avoid errors when running job.result()
quantum_instance._qjob_config = {}

In [18]:
# Create optimizer with only one iteration
optimizer = SPSA(maxiter=1)
# Create the variational form of the ansatz
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")
# Create a VQE object that runs the algorithm
vqe = VQE(qubit_op, var_form, optimizer=optimizer)
# Set the quantum instance to be able to run only the last iteration
vqe.quantum_instance = quantum_instance

The below cell will evaluate the energy at `p0` using the Quantinuum emulator.

You have to add the molecular nuclear repulsion energy to the final result to get the ground state of the molecule.

In [19]:
vqe._energy_evaluation(parameters=p0) + molecule.nuclear_repulsion_energy

.....................

-1.1198872295256754


In this notebook, you've run a single iteration of VQE on an Azure Quantum backend to calculate the ground state of a $H_2$ molecule. Nice job! 👏🏽

As a next step, you can modify the sample to run your own molecule, or run it on hardware.